In [ ]:
import pickle
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'Interpret-master')

from dataset import NewsDataset
from model import DistilBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient
from saliency_interpreter import SaliencyInterpreter

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer

from IPython.display import display, HTML

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

In [ ]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
distilbert = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
distilbert.to(device)

criterion = nn.CrossEntropyLoss()

batch_size = 1

In [ ]:
textl = """"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." MARY yelled as she continued to yell and jump about.
 "Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. "Wow your lighter 
 then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes."""
textr = """"Are you okay?" His simple inquiry about her 
 farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment. "What do you think?" her face now  the 
evidence of the strain she had been carrying for weeks. Her separation from Marcos made her achy and distressed. Her heart was breaking into a million
 pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

text = textl + " [SEP] " + textr


t = tokenizer(text, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

In [ ]:
distilbert.eval()
with torch.no_grad():
    output = distilbert(**t)

In [ ]:
ids = t[0].ids
type_ids = t[0].type_ids
attention_mask = t[0].attention_mask

In [ ]:
input = {'input_ids': torch.tensor(ids), 'token_type_ids': torch.tensor(type_ids), 'attention_mask': torch.tensor(attention_mask), 'labels': torch.tensor(1)}
input

In [ ]:
dataloader = DataLoader(
    [input],
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
test_example = [
    ["Interpretation of HuggingFase's model decision"], 
    ["Transformer-based models have taken a leading role "
     "in NLP today."]
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=256, 
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
distilbert.to('cpu')
distilbert.eval()
with torch.no_grad():
    for batch in dataloader:
        #print(batch)

        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = distilbert(**batch, output_hidden_states=True)
        cur_logits = outputs.logits
        loss = outputs.loss
        print(outputs, type(outputs), loss)

In [ ]:
integrated_grad = IntegratedGradient(
    distilbert, 
    criterion, 
    tokenizer, 
    show_progress=False,
    encoder="bert"
)
instances = integrated_grad.saliency_interpret(dataloader)

In [ ]:
test_example = [
    ["Interpretation of HuggingFase's model decision"], 
    ["Transformer-based models have taken a leading role "
     "in NLP today."]
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=config.max_position_embeddings, 
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
import transformers
import shap

# load a transformers pipeline model
model = transformers.pipeline('sentiment-analysis', return_all_scores=True)

# explain the model on two sample inputs
explainer = shap.Explainer(model) 
shap_values = explainer(["What a great movie! ...if you have no taste."])

# visualize the first prediction's explanation for the POSITIVE output class
shap.plots.text(shap_values[0, :, "POSITIVE"])

In [ ]:
segm00 = """She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, she reached down and pulled out the spare key Joey kept there for her.
 She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs into his room. She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house.
  "MARY!" Joey yelled, standing dumb-struck in the doorway. "Mary? Mary are you okay? Mary whats wrong?" Joey asked, crossing the room and trying to pry the pillow from her clutches. "Joey?" Mary blubbered. 
  "You aren"t supposed to be back for another two days.[SEP]"Joh..n." her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. They jerked apart 
  and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. Then she bit out,"Nice going laser. As usual I"ll have to fix your mess." 
  Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it. Marcos lifted his palms, blinding yellow light emitted from his palms and welded the metal into the floor, 
  filling the hole. With a tortured glance at Lorna he remarked, "I fix my own messes. You don"t have to worry about them from now on." Then he sauntered away."""

In [ ]:
segm00short = """She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, 
she reached down and pulled out the spare key Joey kept there for her. She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs into his room. 
She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house. "MARY!" Joey yelled, standing dumb-struck in the doorway.[SEP]"Joh..n." 
her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. They jerked apart and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. 
Then she bit out,"Nice going laser. As usual I"ll have to fix your mess." Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it."""

In [ ]:
segm10 = """"DNA?" Javier asked. The ME met his gaze, eyes twinkling. "Oh yeah. And lots of it." "Oh, I"m listening," Ryan said with a grin. "Well, there were male epithelials under her fabulously manicured nails, 
but I"ve got at least three other donors." Ryan and Javier exchanged a look. "Define donors," Ryan requested. "I"ve got saliva and semen," Lanie replied, checking the notes in the folder. "Multiple donors for both too. 
From the looks of things our girl had multiple sexual partners just before she died." Ryan"s eyebrow went up. "Are you telling us this girl was in an orgy?" "Dude, Castle would love this," Javier couldn"t help exclaiming. 
He shook his head as he and Ryan exchanged a look.[SEP]"Are you insane?" She"d subconsciously taken a step closer to him. However, he mirrored her step backwards. "I"m not finished. You deserve more answers than that." 
This new found honesty shocked Sara enough to keep her mouth shut. However, he stopped, rubbing a hand over his face and sighing, partly in anger. "Do you have any idea how much danger you"re putting yourself in?" he
 whispered to her, taking another step away and increasing the distance. "When you didn"t know, you could tell the truth. I don"t want you to lie, Sara." They both knew it was pointless. He"d involved her in his plan from day one.
  That"s when it all started to slip into her mind, puzzle pieces falling snugly into place. "I was a tool.'"""

In [ ]:
import shap
import torch
import transformers

model = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
tokenizer = transformers.AutoTokenizer.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")

#model = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results4\checkpoint-180000")
#tokenizer = transformers.AutoTokenizer.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results4\checkpoint-180000")

def f(pairs):
    outs = []
    for q in pairs:
        text1, text2 = q.split("[SEP]")
        d = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')
        with torch.no_grad():
            out = model(**d)
        logits = out.logits
        outs.append(logits.reshape(-1).detach().numpy())
    return outs

# attach a dynamic output_names property to the models so we can plot the tokens at each output position
def out_names(inputs):
    text1, text2 = inputs.split("[SEP]")
    d = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=255)
    return [tokenizer.decode([id]) for id in d["input_ids"]]

f.output_names = out_names

In [ ]:
# load results of LIME analysis
import pandas as pd
respd = pd.read_csv("res_100.csv")
texts = respd['text']

In [ ]:
explainer = shap.Explainer(f, tokenizer)

In [ ]:
shaps = []
for index, text in enumerate(texts):
    shaptext = text.replace("$&*&*&$", "[SEP]")
    explainer = shap.Explainer(f, tokenizer)
    shap_values = explainer([shaptext])
    shaps.append([shaptext, shap_values])
    

In [ ]:
show_shaps(shaps, 0, 10)

In [ ]:
def show_shaps(shaps, start, end):
    cur_shaps = shaps[start:end]
    for i, exp in enumerate(cur_shaps): 
        exp_label = int(respd['label'][i])
        print("Explanation #{}, label {}".format(i, exp_label))
        shap.plots.text(exp[1][0, :, exp_label])

In [ ]:
import numpy as np

shaps[1].base_values[0]

label = np.argmax(shaps[1].base_values[0])

In [ ]:
for i in (1,3,5):
    shap.plots.text(shaps[i][0, :, 0])

In [ ]:
prediction=pipe([segm00])
print(prediction[0])
type(prediction[0][0])

In [ ]:
a = f2([segm00])
a

In [ ]:
for id in tokenizer("JOHN")["input_ids"]:
    print(tokenizer.decode([id])) 

In [ ]:
segm1 = """Rinoa let out a soft giggle. "Okay Uncle Laguna." "As always, make yourselves at home!" Kiros cleared his throat. "Laguna, I believe our guests are hungry." "OH! Yes yes, I"m sorry." 
Laguna scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed Laguna as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one, though the lenghty table was pleasingly decorated with foods on the table.[SEP]The crown prince already entreated help from the glaives, a last resort he would 
rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none.
 It was when Aranea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""
segm2 = """Rinoa noticed how there were so much dishes served, enough to feed one whole army. "Please, take a seat." Laguna gestured at them as he sat at a chair placed at the end of the long table. He then looked around, as if looking for someone. 
"Your son said he"ll be joining us in a few minutes." Kiros plainly said. "Ah..." Laguna gave out a sheepish smile. "Well, can"t let our guests wait too long, can we? C"mon now, gobble up! Don"t be shy, just eat...[SEP]She even almost succeeded
 at her own assassination attempt at the crown prince, only to pull back and used a lame excuse as "I do not do overtimes." when the battle ran for too long. 
But she did also express her intentions to leave her post as their hired mercenary afterwards when things got too senseless and chaotic with the Imperials. She was only in for the job and money, not for some loyalty or building a good name in her trade."""

In [ ]:
segm3 = """"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." Mary yelled as she continued to yell and jump about. 
"Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. 
"Wow your lighter then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.[SEP]"Are you okay?" 
His simple inquiry about her farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment.
 "What do you think?" her face now bore the evidence of the strain she had been carrying for weeks. 
Her separation from Marcos made her achy and distressed. Her heart was breaking into a million pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

In [ ]:
segm4 = """What hospital are you at?" Dustin said and there was a lot of rustling at the other end. "The one near my house. I didn"t catch the name I was a little occupied. 
Mary was throwing a fit." Joey answered sincerely. "Kay, we are coming. See you in a few." Dustin said and the line went dead. Joey took a seat along the wall and waited for news about Mary.
 Just a tad bit of drama, hope its not too confusing. Blondie : P Oh gosh, sorry. I left you a cliff hanger. I wonder what will happen to Mary today.[SEP]She bobbed her head slightly from side to side and with a sheepish look remarked,
  "Not exactly but something to the effect. I did call him a ninny though." John flashed his killer smile, it made her gooey every time. In an instant his eyes turned intense again and he rested his forehead on hers. The change in him didn"t go unnoticed by Clarice. 
"What?" "Clarice I...feel it again." He had told her about how he sometimes felt a foreboding feeling. "Hey."""

In [ ]:
segm3_altered = """"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." Mary yelled as she continued to yell and jump about. 
"Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. 
"Wow your lighter then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.[SEP]"Are you okay?" 
His simple inquiry about her farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment.
 "What do you think?" her face now  the evidence of the strain she had been carrying for weeks. 
Her separation from Marcos made her achy and distressed. Her heart was breaking into a million pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

In [ ]:
segm4_altered = """What hospital are you at?" Dustin said and there was a lot of rustling at the other end. "The one near my house. I didn"t catch the name I was a little occupied. 
Mary was throwing a fit." Joey answered sincerely. "Kay, we are coming. See you in a few." Dustin said and the line went dead. Joey took a seat along the wall and waited for news about Mary.
 Just a tad bit of drama, hope its not too confusing. Blondie : P Oh gosh, sorry. I left you a cliff hanger. I wonder what will happen to Mary today.[SEP]She bobbed her head slightly from side to side and with a sheepish look remarked,
  "Not exactly but something to the effect. I did call him a ninny though." John flashed his killer smile, it made her gooey every time. In an instant his eyes turned intense again and he rested his forehead on hers. The change in him didn"t go unnoticed by . 
"What?" " I...feel it again." He had told her about how he sometimes felt a foreboding feeling. "Hey."""

In [ ]:
segm1_corrected = """Rinoa let out a soft giggle. "Okay Uncle Laguna." "As always, make yourselves at home!" Kiros cleared his throat. "Laguna, I believe our guests are hungry." "OH! Yes yes, I"m sorry." 
Laguna scratched the back of his neck in embarrassment. "To the dining hall we go!" They all followed Laguna as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one, though the lengthy table was pleasingly decorated with foods on the table.[SEP]The crown prince already entreated help from the glaives, a last resort he would 
rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none.
 It was when Aranea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""

In [ ]:
segm4433 = """" Sir, I have the the reports you"ve asked for" The bridge commander stated. "
 Ahhh, thank you commander. Prep the fleet for the jump to Kashyyik" Admiral Segutav ordered in 
 his thick German accent. " Yessir. Setting location now" The commander replied. " if I may ask sir, 
 why did you need the reports on the Ghost"s crew?" " I"ve been put in charge of this fleet by none other than 
 Grand Admiral Thrawn himself. He put me in command of this fleet with the sole purpose to destroy the Phoenix
  Squadron.[SEP]Hercules woke up when he heard Xena dreaming so he went and fetched some ale and put some
   sleeping poison in it. "Xena, here take a drink." Hercules kindly demanded. "Thank you, I think I need a drink." 
   Xena replied. "Xena, what was that all about?" Hercules asked. "I had a dream I was on Mount Olympus and Ares jumped 
   out from somewhere with Gabrielle, 
then we started talking and all of a sudden he killed Gabrielle." Xena sobbed."""

In [ ]:
data = [segm4433]

explainer_start = shap.Explainer(f, tokenizer)
shap_values = explainer_start(data)

In [ ]:
shap.plots.text(shap_values[0, :, 0])

In [ ]:
data = [segm3_altered]

explainer_start = shap.Explainer(f, tokenizer)
shap_values3 = explainer_start(data)

In [ ]:
data = [segm4]

explainer_start = shap.Explainer(f, tokenizer)
shap_values4 = explainer_start(data)

In [ ]:
data = [segm4_altered]

explainer_start = shap.Explainer(f, tokenizer)
shap_values41 = explainer_start(data)

In [ ]:
data = [segm00]

explainer_start = shap.Explainer(f, tokenizer)
shap_values_long_00 = explainer_start(data)

In [ ]:
shap.plots.text(shap_values_long_00[0, :, 0])
#short, regular model

In [ ]:
shap.plots.text(shap_values_long_00[0, :, 0])
#long, big model 180k

In [ ]:
shap.plots.text(shap_values_long_00[0, :, 0])
#long, big model 450k

In [ ]:
shap.plots.text(shap_values_long_00[0, :, 0])
#long, big model 450k

In [ ]:
# local explanation for your first example
shap.plots.text(shap_values_start[0, :, 1])
# global summary explanation

In [ ]:
shap.plots.text(shap_values[0, :, 1])
#corrected


In [ ]:
shap.plots.text(shap_values[0, :, 1])

In [ ]:
shap.plots.text(shap_values[0, :, 1])

In [ ]:
shap.plots.text(shap_values[0, :, 0])

In [ ]:
shap.plots.text(shap_values3[0, :, 0])

In [ ]:
shap.plots.text(shap_values4[0, :, 1])

In [ ]:
shap.plots.text(shap_values41[0, :, 1])

In [ ]:
shap.plots.text(shap_values[0, :, 1])

In [ ]:
shap_values

In [ ]:
shap_values.shape

In [ ]:
shap_values_couplt.data[0]

In [ ]:
shap.plots.waterfall(shap_values[0, :, 0])

In [ ]:
shap.plots.bar(shap_values[0, :, 0], max_display=20, order=shap.Explanation.argsort.flip)

In [ ]:
shap.plots.bar(shap_values_start[0])

In [ ]:
shap_values_start